# PageRank

In this problem, you have to implement PageRank algorithm to rank papers based on their references.

PageRank is a link analysis algorithm and it assigns a numerical weighting to each element of a hyperlinked set of documents, such as the World Wide Web, with the purpose of "measuring" its relative importance within the set. The algorithm may be applied to any collection of entities with reciprocal quotations and references.

It's recommended to have a look at the following sources to get more familiar with PageRank:

[https://en.wikipedia.org/wiki/PageRank](https://en.wikipedia.org/wiki/PageRank)

[https://towardsdatascience.com/pagerank-3c568a7d2332](https://towardsdatascience.com/pagerank-3c568a7d2332)

## Dataset
You are given a number of papers in Computer Science field, which are crawled from [Semantic Scholar](https://www.semanticscholar.org/). Each paper has Id, title, references, etc. You can download the dataset from here. For your convenience, we have considered only a limited number of references for each paper.


## Hint
Each paper is a node in the graph. Paper A links to paper B if and only if B is in A's references. (Similary, we have a directed edge from A to B.) Note that some papers may not have any input or output edge. Don't forget to consider such papers as well.

### Using libraries such as networkx is not allowed (Except in the last part). You have to implement PageRank from scratch. Feel free to add cells when needed.   


## Download data, import dependencies

In [1]:
# Download resources https://drive.google.com/drive/folders/1GvUc06eKX2Knf3JP5RCvIJjUkIjE2fTu?usp=share_link
!mkdir -p resources
%cd ./resources
!gdown 1C9l4uWzABZomkZQdAMxETz-b12PcOzBv # clean_data.json
%cd ..

/content/resources
Downloading...
From: https://drive.google.com/uc?id=1C9l4uWzABZomkZQdAMxETz-b12PcOzBv
To: /content/resources/clean_data.json
100% 23.3M/23.3M [00:00<00:00, 171MB/s]
/content


In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
import networkx as nx
import requests
from time import sleep
tqdm.pandas()

In [2]:
df = pd.read_json('resources/clean_data.json')
print(df.shape)
df.head()

(7136, 8)


,paperId,title,abstract,year,referenceCount,citationCount,authors,k_references
0,63d8426ba1f51a8525dd19fd8ec92934ec71aea5,A Survey of Data Augmentation Approaches for NLP,Data augmentation has recently seen increased ...,2021,196,117,"[{'authorId': '152913678', 'name': 'Steven Y. ...",[{'paperId': '00ea88920eca898909bd8dd455df25ec...
1,33ec7eb2168e37e3007d1059aa96b9a63254b4da,Beyond Accuracy: Behavioral Testing of NLP Mod...,Although measuring held-out accuracy has been ...,2020,33,386,"[{'authorId': '78846919', 'name': 'Marco Tulio...",[{'paperId': '05dd7254b632376973f3a1b4d39485da...
2,642038c7a49caa9f0ac5b37b01fab5b2b8d981d5,ERASER: A Benchmark to Evaluate Rationalized N...,State-of-the-art models in NLP are now predomi...,2019,75,217,"[{'authorId': '48727916', 'name': 'Jay DeYoung...",[{'paperId': '0754982927fa07a6689fb0f2cbeb8e3d...
3,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,Large pre-trained language models have been sh...,2020,71,339,"[{'authorId': '145222654', 'name': 'Patrick Le...",[{'paperId': '016368185723d0ec99aafa4b59273005...
4,d47a682723f710395454687319bb55635e653105,Language (Technology) is Power: A Critical Sur...,We survey 146 papers analyzing “bias” in NLP s...,2020,238,324,"[{'authorId': '3422038', 'name': 'Su Lin Blodg...",[{'paperId': '00059087c954c1af6ece33115315e3e0...


## PageRank
Implement PageRank from scratch!


*   Don't forget to consider the damping factor in your implementation.
*   Report number of nodes and number of edges of the constructed graph.
*   Identify the node with maximum number of input edges. Which paper corresponds to this node?
*   Report 10 most importatnt papers with PageRank.

See the most important paper according to PageRank. Recall that papers are in CS and NLP topics. What is your opinion about this paper? :)



In [3]:
class Node:
  def __init__(self,paper_id,children=set(),parents=set()):
    self.paper_id=paper_id
    self.children=children
    self.parents=parents
    self.ranking=1
  def __repr__(self):
    return f'PaperId: {self.paper_id}/Ranking: {self.ranking}/Out degree: {len(self.children)}/In degree: {len(self.parents)}'
  def __gt__(self,other):
    return self.ranking>other.ranking

  def add_children(self,children:list):
    self.children=self.children.union(set(children))
  def add_parent(self,parent):
    self.parents=self.parents.union(set([parent]))

class PageRank:
  def __init__(self,graph:list,damping_factor=0.1,num_iterations=1000):
    self.graph=graph
    self.damping_factor=damping_factor
    self.num_iterations=num_iterations
  def process(self):
    for _ in tqdm(range(self.num_iterations)):
      for node in self.graph:
        #update the node rank
        random_walk=self.damping_factor/len(self.graph)
        new_ranking=random_walk+(1-self.damping_factor)*sum(parent.ranking/len(parent.children) for parent in node.parents)
        node.ranking=new_ranking
    

In [4]:
def construct_graph(df:pd.DataFrame):
  '''
  Construct the graph which is a list of nodes each node having
  a set of children and parents
  '''
  nodes_created=dict()
  for i,row in df.iterrows():
    paper_id=row['paperId']
    reference_ids=[paper['paperId'] for paper in row['k_references']]
    reference_nodes=[]
    for reference_id in reference_ids:
      if nodes_created.get(reference_id) is None:
        reference_node=Node(reference_id)
        nodes_created[reference_id]=reference_node
      else:
        reference_node=nodes_created[reference_id]
      reference_nodes.append(reference_node)
    if nodes_created.get(paper_id) is None:
      paper_node=Node(paper_id,reference_nodes)
      nodes_created[paper_id]=paper_node
    else:
      paper_node=nodes_created[paper_id]
      paper_node.add_children(reference_nodes)
    for reference_node in reference_nodes:
      reference_node.add_parent(paper_node)
  return list(nodes_created.values())


graph=construct_graph(df)
print(f'Number of nodes: {len(graph)}, Number of edges: {sum(len(node.parents)+len(node.children) for node in graph)//2}')
print(f'Node with maximum number of input edges: {sorted(graph,key=lambda x:len(x.parents),reverse=True)[0].paper_id}')

Number of nodes: 40203, Number of edges: 62513
Node with maximum number of input edges: 204e3073870fae3d05bcbc2f6a8e263d9b72e776


In [5]:
page_rank=PageRank(graph=graph,damping_factor=0.13,num_iterations=600)
page_rank.process()
ten_best_nodes=sorted(graph,reverse=True)[:10]
print('Ten most important papers are: '+', '.join([f'{n.paper_id}' for n in ten_best_nodes]))

100%|██████████| 600/600 [00:48<00:00, 12.47it/s]


Ten most important papers are: 204e3073870fae3d05bcbc2f6a8e263d9b72e776, 0b44fcbeea9415d400c5f5789d6b892b6f98daff, 05dd7254b632376973f3a1b4d39485da17814df5, 0b544dfe355a5070b60986319a3f51fb45d1348e, 077f8329a7b6fa3b7c877a57b81eb6c18b5f87de, 44d2abe2175df8153f465f6c39b68b76a0d40ab9, 1af68821518f03568f913ab03fc02080247a27ff, 330da625c15427c6e42ccfa3b747fb29e5835bf0, 2c03df8b48bf3fa39054345bafabfeff15bfd11d, 084c55d6432265785e3ff86a2e900a49d501c00a


In [8]:
best_paper=ten_best_nodes[0]
print(best_paper)
paper=request_papers_by_id([best_paper.paper_id])
print(paper)

PaperId: 204e3073870fae3d05bcbc2f6a8e263d9b72e776/Ranking: 0.00016839735169589895/Out degree: 0/In degree: 517


100%|██████████| 1/1 [00:00<00:00,  2.32it/s]

[{'paperId': '204e3073870fae3d05bcbc2f6a8e263d9b72e776', 'url': 'https://www.semanticscholar.org/paper/204e3073870fae3d05bcbc2f6a8e263d9b72e776', 'title': 'Attention is All you Need', 'year': 2017, 'referenceCount': 39, 'citationCount': 48352, 'fieldsOfStudy': ['Computer Science']}]


As you can see in the previous cell the most important paper title is 'Attention is All you Need', and according to the semantic scholar it is a paper about Attention in CNNs and RNNs and it has about 48000 citations and is considered as a highly influential paper.

## Networkx
Implement PageRank with networkx. Report previous items and compare the results with your implementation. Explain if there is any differrence.

In [20]:
G=nx.DiGraph()
G.add_nodes_from(graph)
G.add_edges_from([(node,child) for node in graph for child in node.children])
pr=nx.pagerank(G,alpha=0.9)
networkx_ten_best_nodes=sorted(pr,reverse=True)[:10]

In [22]:
networkx_ten_best_nodes

[PaperId: 204e3073870fae3d05bcbc2f6a8e263d9b72e776/Ranking: 0.00016839735169589895/Out degree: 0/In degree: 517,
 PaperId: 0b44fcbeea9415d400c5f5789d6b892b6f98daff/Ranking: 0.00010580223044356603/Out degree: 0/In degree: 280,
 PaperId: 05dd7254b632376973f3a1b4d39485da17814df5/Ranking: 7.792638415085937e-05/Out degree: 0/In degree: 189,
 PaperId: 0b544dfe355a5070b60986319a3f51fb45d1348e/Ranking: 7.4846280250507e-05/Out degree: 0/In degree: 208,
 PaperId: 077f8329a7b6fa3b7c877a57b81eb6c18b5f87de/Ranking: 7.461711382053592e-05/Out degree: 10/In degree: 227,
 PaperId: 44d2abe2175df8153f465f6c39b68b76a0d40ab9/Ranking: 7.373847841100491e-05/Out degree: 0/In degree: 203,
 PaperId: 1af68821518f03568f913ab03fc02080247a27ff/Ranking: 6.321675778194444e-05/Out degree: 0/In degree: 170,
 PaperId: 330da625c15427c6e42ccfa3b747fb29e5835bf0/Ranking: 5.99324510764591e-05/Out degree: 0/In degree: 176,
 PaperId: 2c03df8b48bf3fa39054345bafabfeff15bfd11d/Ranking: 5.09040631406416e-05/Out degree: 0/In degree

In [21]:
ten_best_nodes

[PaperId: 204e3073870fae3d05bcbc2f6a8e263d9b72e776/Ranking: 0.00016839735169589895/Out degree: 0/In degree: 517,
 PaperId: 0b44fcbeea9415d400c5f5789d6b892b6f98daff/Ranking: 0.00010580223044356603/Out degree: 0/In degree: 280,
 PaperId: 05dd7254b632376973f3a1b4d39485da17814df5/Ranking: 7.792638415085937e-05/Out degree: 0/In degree: 189,
 PaperId: 0b544dfe355a5070b60986319a3f51fb45d1348e/Ranking: 7.4846280250507e-05/Out degree: 0/In degree: 208,
 PaperId: 077f8329a7b6fa3b7c877a57b81eb6c18b5f87de/Ranking: 7.461711382053592e-05/Out degree: 10/In degree: 227,
 PaperId: 44d2abe2175df8153f465f6c39b68b76a0d40ab9/Ranking: 7.373847841100491e-05/Out degree: 0/In degree: 203,
 PaperId: 1af68821518f03568f913ab03fc02080247a27ff/Ranking: 6.321675778194444e-05/Out degree: 0/In degree: 170,
 PaperId: 330da625c15427c6e42ccfa3b747fb29e5835bf0/Ranking: 5.99324510764591e-05/Out degree: 0/In degree: 176,
 PaperId: 2c03df8b48bf3fa39054345bafabfeff15bfd11d/Ranking: 5.09040631406416e-05/Out degree: 0/In degree

As you can see in the previous cells, our implementation of PageRank gives us the same results as the networkx ready pagerank function for the best 10 ranking pages.

## Utils
Below is the main function we used to get the papers, in case you were wondering. You may want to use it to get more information about the papers.

In [7]:
# fields are separated by ",". For more information see https://api.semanticscholar.org/api-docs/graph
def request_papers_by_id(IDs, fields='title,url,year,fieldsOfStudy,citationCount,referenceCount'):
    papers = []
    for id in tqdm(IDs):
        response = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/{id}?fields={fields}')
        js = response.json()
        papers.append(js)
        # sleep(3.1)
    return papers